# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Описание данных

Признаки:

- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с- .)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

Целевой признак

- Price — цена (евро)

In [1]:
# ОТ РЕВЬЮЕРА
!pip install ydata_profiling
!pip install colorama
!pip install category_encoders
!pip install pandas --upgrade
!pip install catboost --upgrade


# для анализа и работы с данными
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import plotly.express as px
from scipy import stats
import plotly.express as px
import pandas as pd
from scipy.stats import norm
from colorama import Fore, Style

# алгоритмы машинного обучения
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint
import requests
from urllib.parse import urlencode
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
import warnings
from sklearn.compose import ColumnTransformer
warnings.filterwarnings("ignore")
from sklearn.ensemble import BaggingRegressor
from sklearn.pipeline import (
                Pipeline, 
                make_pipeline
)
from sklearn.model_selection import ( 
                train_test_split,
                GridSearchCV,
                cross_val_score
)
from category_encoders import TargetEncoder

# метрики для алгоритмов ML
from sklearn.metrics import mean_squared_error

# счетчик времени
import time

# константа для алгоритмов ML
RANDOM_STATE = 87

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:


def plot_column_values(df, column_name, y=None, plot_type='histogram', bins=None, color=None, x_label=None, y_label=None, title=None):
    """
    Функция для отображения значений столбца с помощью интерактивных графиков.
    
    Аргументы:
    - df: DataFrame, исходный DataFrame
    - column_name: str, название столбца, значения которого нужно отобразить
    - plot_type: str, тип графика ('histogram' - гистограмма, 'box' - боксплот, 'line' - линейный график, 'pie' - круговая диаграмма, 'violin' - ящик с усами, 'scatter' - точечный график, 'density_contour' - график плотности с контурами, 'density_heatmap' - график плотности в виде тепловой карты)
    - bins: int, количество бинов (только для гистограммы)
    - color: str, цвет графика (только для гистограммы и боксплота)
    - x_label: str, подпись оси x
    - y_label: str, подпись оси y
    - title: str, заголовок графика
    """
    if plot_type == 'histogram':
        fig = px.histogram(df, y=y,x=column_name, nbins=bins, title=title, color=color)
    elif plot_type == 'box':
        fig = px.box(df, y=column_name, title=title, color=color)
    elif plot_type == 'line':
        fig = px.line(df, x=df.index, y=column_name, title=title, color=color)
    elif plot_type == 'pie':
        fig = px.pie(df, names=column_name, title=title, color=color)
    elif plot_type == 'violin':
        fig = px.violin(df, y=column_name, title=title, color=color)
    elif plot_type == 'scatter':
        fig = px.scatter(df, x=df.index, y=column_name, title=title, color=color)
    elif plot_type == 'density_contour':
        fig = px.density_contour(df, x=df.index, y=column_name, title=title, color=color)
    elif plot_type == 'density_heatmap':
        fig = px.density_heatmap(df, x=df.index, y=column_name, title=title, color_continuous_scale='Blues')
    else:
        raise ValueError("Недопустимый тип графика. Допустимые значения: 'histogram', 'box', 'line', 'pie', 'violin', 'scatter', 'density_contour' или 'density_heatmap'.")
    
    fig.update_layout(xaxis_title=x_label, yaxis_title=y_label)
    fig.show()
    
    
def get_sorted_unique_values(df, column_name):
    """
    Функция для получения отсортированного списка уникальных значений столбца.
    
    Аргументы:
    - df: DataFrame, исходный DataFrame
    - column_name: str, название столбца
    
    Возвращает:
    - sorted_unique_values: list, отсортированный список уникальных значений столбца
    """
    unique_values = df[column_name].unique()
    sorted_unique_values = pd.Series(unique_values).sort_values().tolist()
    
    return sorted_unique_values

def get_info(df):
    """
    функция выводит общую статистику по датафрейму и отрисовывает графики по каждому признаку:
    - для признаков типа int гистограмму
    - для признаков типа object круговую диаграмму
    - для признаков с количеством значений > 10000 шт. - ничего
    """
    
    s_br, s_reset = Style.BRIGHT, Style.RESET_ALL
    display(df.head(3), df.tail(3), df.describe().T)

    num_cols, cat_cols = [], []
    for num, i in enumerate(df.columns):
        if pd.api.types.is_numeric_dtype(df[i]):
            print(f'{num+1}) {s_br}{i}{s_reset} (тип int), оригинальных значений: {len(df[i].unique())}')
            num_cols.append(i)
        elif pd.api.types.is_object_dtype(df[i]):
            print(f'{num+1}) {s_br}{i}{s_reset} (тип object), оригинальных значений: {len(df[i].unique())}')
            cat_cols.append(i)
    print(f'\n{s_br}итого:{s_reset}\nтип int: {len(num_cols)}\nтип object: {len(cat_cols)}\n'
          f'\n{s_br}пропуски:{s_reset}\n{df.isna().sum()}\n\n{s_br}дубликатов:{s_reset} {df.duplicated().sum()}')
    
    fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(10, 8))
    fig.suptitle('распределение значений признаков', fontsize=16, fontweight='bold')

    for num, col_name in enumerate(df.columns):
        row = num // 4
        col = num % 4
        if df[col_name].nunique() > 10000: 
            ax[row][col].set_title(col_name)
            ax[row][col].set_xticks([])
            ax[row][col].set_yticks([])
            continue
        if col_name in num_cols: 
            ax[row][col].hist(df[col_name].dropna(), color='#4C72B0', edgecolor='#5AB3CB', bins=20)
            ax[row][col].set_xticks([])
            ax[row][col].set_yticks([])
        else:
            counts = df[col_name].dropna().value_counts()
            ax[row][col].pie(counts.values.tolist(), startangle=90, 
                             wedgeprops={'linewidth': .7, 'edgecolor': 'white'})
        ax[row][col].set_title(col_name)
    plt.subplots_adjust(hspace=.5, wspace=.3);
    
def fill_mode(group, column_name):
    """функция заполняет пропуски модой группы"""
    
    if group[column_name].notna().any():
        mode = stats.mode(group[column_name].dropna())[0][0]
        group[column_name].fillna(mode, inplace=True)
    return group

def count_missing_values_percentage(df):
    """
   функция выводит количество пропусков в столбцах, и процент от общего количества.
    """
    missing_values = df.isnull().sum()
    total_values = df.shape[0]
    missing_percentage = (missing_values / total_values) * 100
    missing_data = pd.concat([missing_values, missing_percentage], axis=1, keys=['Missing Values', 'Missing Percentage'])
    print(missing_data)


# Подготовка данных

## Загрузка данных

In [4]:

try:
    df = pd.read_csv('autos.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')


## Изучаем данные

In [5]:
display(df.head())
display(df.tail())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
354368,2016-03-20 19:41:08,3400,wagon,2002,manual,100,golf,150000,6,gasoline,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


In [6]:
#display(ProfileReport(df))

Изучив данные напишем список задач для подготовки данных:

- Удалить не нужные столбцы 'DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures
- Перевести столбцы в нижний регистр в стиле змейки.
- Удалить дублирующие строки
- Удалить выбросы строк по годам
- Удалить выбросы строк по цене
- Удалить выбросы строк по мощности
- Дополним данные названием стран, городами, названием областей и кординат используя почтовый индекс.
- Заполним пропуски модой в столбцах Gearbox, VehicleType, Model, Repaired, FuelType групируя по бренду, модели, и году выпуска
- Заполним пропуски в столбце Repaired словом unknow так как не можем себе позволить удалить 70 000 строк, а заполнять его групируя по другим признакам бесмысленно.
- Перевести категориальные столбцы в числовые используя One-Hot Encoding



## Подготовка данных

### Удалить не нужные столбцы 'DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures

In [7]:
# Список столбцов для удаления
columns_to_drop = ['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures']

# Удаляем столбцы из DataFrame
df.drop(columns=columns_to_drop, inplace=True)

### Перевести столбцы в нижний регистр в стиле змейки

Переменуем столбцы

In [8]:
df = df.rename(
 columns={
'Price': 'price',
'VehicleType': 'vehicle_type',
'RegistrationYear': 'registration_year',
'Gearbox': 'gearbox',
'Power': 'power',
'Model': 'model',
'Kilometer': 'kilometer',
'RegistrationMonth': 'registration_month',
'FuelType': 'fuel_type',
'Brand': 'brand',
'Repaired': 'repaired',
'PostalCode': 'postal_code',
 }
)


In [9]:
df

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,postal_code
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,70435
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,66954
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,90480
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,60437
...,...,...,...,...,...,...,...,...,...,...,...,...
354364,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2694
354365,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,39576
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,26135
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,87439


### Удалить дублирующие строки

In [10]:
df = df.drop_duplicates().reset_index(drop=True)

### Удалить выбросы строк по годам

In [11]:
#получениt отсортированного списка уникальных значений столбца registration_year
display(get_sorted_unique_values(df, 'registration_year'))

[1000,
 1001,
 1039,
 1111,
 1200,
 1234,
 1253,
 1255,
 1300,
 1400,
 1500,
 1600,
 1602,
 1688,
 1800,
 1910,
 1915,
 1919,
 1920,
 1923,
 1925,
 1927,
 1928,
 1929,
 1930,
 1931,
 1932,
 1933,
 1934,
 1935,
 1936,
 1937,
 1938,
 1940,
 1941,
 1942,
 1943,
 1944,
 1945,
 1946,
 1947,
 1948,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2066,
 2200,
 2222,
 2290,
 2500,
 2800,
 2900,
 3000,
 3200,
 3500,
 3700,
 3800,
 4000,
 4100,
 4500,
 4800,
 5000,
 5300,
 5555,
 5600,
 5900,
 5911,
 6000,
 6500,
 7000,
 7100,
 7500,
 7800,
 8000,
 8200,

In [12]:
#plot_column_values(df, 'registration_year', plot_type='histogram', bins=df['registration_year'].nunique(), x_label='Год регистрации', y_label='Количество', title="Распределение года регистрации")

Избавимся от некоректных годов

In [13]:
# Удаление строк, где registration_year < 1980 или registration_year > 2016
df = df.drop(df[(df['registration_year'] < 1900) | (df['registration_year'] > 2016)].index)

In [14]:
#plot_column_values(df, 'registration_year', plot_type='histogram', bins=df['registration_year'].nunique(), x_label='Год регистрации', y_label='Количество', title="Распределение года регистрации")

Значений до 1980 очень мало, основное количество машин в нашем дадасете от 1995 до 2010

In [15]:
# Удаление строк, где registration_year < 1980 или registration_year > 2016
df = df.drop(df[(df['registration_year'] < 1900) | (df['registration_year'] > 2016)].index)

### Удалить выбросы строк по цене

In [16]:
#plot_column_values(df, 'price', plot_type='histogram', bins=100, x_label='Цена автомобиля', y_label='Количество', title="Распределение цен авто")

In [17]:
#plot_column_values(df, 'price', plot_type='box', bins=100, y_label='Цена автомобиля', title="Распределение цен авто")

In [18]:
#получениt отсортированного списка уникальных значений столбца price
display(get_sorted_unique_values(df, 'price'))

[0,
 1,
 2,
 3,
 4,
 5,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 24,
 25,
 26,
 27,
 29,
 30,
 32,
 33,
 35,
 38,
 39,
 40,
 45,
 47,
 49,
 50,
 55,
 58,
 59,
 60,
 65,
 66,
 69,
 70,
 74,
 75,
 77,
 79,
 80,
 85,
 88,
 89,
 90,
 95,
 98,
 99,
 100,
 101,
 105,
 108,
 110,
 111,
 112,
 115,
 117,
 119,
 120,
 121,
 122,
 123,
 125,
 126,
 127,
 128,
 129,
 130,
 132,
 133,
 135,
 139,
 140,
 142,
 145,
 149,
 150,
 155,
 156,
 157,
 158,
 159,
 160,
 162,
 165,
 166,
 169,
 170,
 173,
 175,
 177,
 179,
 180,
 181,
 185,
 188,
 189,
 190,
 192,
 193,
 195,
 196,
 198,
 199,
 200,
 202,
 205,
 209,
 210,
 211,
 215,
 217,
 219,
 220,
 222,
 224,
 225,
 229,
 230,
 233,
 235,
 236,
 238,
 240,
 243,
 245,
 248,
 249,
 250,
 251,
 252,
 253,
 255,
 259,
 260,
 261,
 263,
 265,
 266,
 269,
 270,
 272,
 273,
 274,
 275,
 277,
 278,
 279,
 280,
 281,
 284,
 285,
 288,
 289,
 290,
 293,
 295,
 298,
 299,
 300,
 301,
 305,
 308,
 309,
 310,
 315,
 316,
 319,
 320

In [19]:
#удаляем строки где цена меньше 100 евро
df = df.drop(df[(df['price'] < 100)].index)

### Удалить выбросы строк по мощности

In [20]:
display(get_sorted_unique_values(df, 'power'))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [21]:
#plot_column_values(df, 'power', plot_type='box', bins=100, y_label='Количество', title="Распределение мощности")

In [22]:
#plot_column_values(df, 'power', plot_type='histogram', bins=100, x_label='Мощность', y_label='Количество', title="Распределение мощности")

In [23]:
#удаляем строки где мощность больше 550 л\c
df = df.drop(df[(df['power'] > 550)].index)

Исправил

In [24]:
#Отоброжаем 20 случайных строк где в столбце power значение равно 0
random_rows = df[df['power'] == 0].sample(n=20, random_state=RANDOM_STATE)
print(random_rows)


        price vehicle_type  registration_year gearbox  power     model  \
49238     800          NaN               2000     NaN      0       NaN   
75068    3400          NaN               2005    auto      0       NaN   
289223   1100        small               2004    auto      0    fortwo   
176038   1800        small               2001    auto      0    fortwo   
194532    800          NaN               2000     NaN      0    kangoo   
174625    200          NaN               2000     NaN      0     swift   
247815    400        small               2002  manual      0     other   
291485   1250        small               2005  manual      0      polo   
81538   20000        sedan               1950  manual      0       NaN   
91246    1950        sedan               1999    auto      0  e_klasse   
78885     750        small               1997  manual      0    fiesta   
257928   1700        sedan               1998  manual      0       5er   
262627   1700        sedan            

In [25]:
count_zero = df['power'].value_counts()[0]

# Выводим результат
print("Количество значений равных 0 в столбце 'power':", count_zero)

Количество значений равных 0 в столбце 'power': 30393


Заполним столбец power медианным группируя по бренду, модели, и году регистрации

In [26]:
df['power'] = df.groupby(['brand', 'model', 'registration_year'])['power'].transform(lambda x: x.replace(0, np.nan).fillna(x.median()))


In [27]:
count_zero = (df['power'] == 0).sum()
print("Количество значений равных 0 в столбце 'power':", count_zero)


Количество значений равных 0 в столбце 'power': 784


А теперь только по бренду и модели

In [28]:
df['power'] = df.groupby(['brand', 'model'])['power'].transform(lambda x: x.replace(0, np.nan).fillna(x.median()))


In [29]:
count_zero = (df['power'] == 0).sum()
print("Количество значений равных 0 в столбце 'power':", count_zero)


Количество значений равных 0 в столбце 'power': 1


Осталось 9 пропусков, с ними разберемся далее.

In [30]:
#plot_column_values(df, 'power', plot_type='histogram', bins=100, x_label='Мощность', y_label='Количество', title="Распределение мощности")

### Добавим новые признаки

В первую очередь добавим категориальный признак цены с шагом 500 евро

In [31]:

bins = list(range(0, 20001, 500))
labels = ['{}-{}'.format(bins[i], bins[i+1]) for i in range(len(bins)-1)]

# Создаем новый столбец с категориальным признаком "price_category"
df['price_category'] = pd.cut(df['price'], bins=bins, labels=labels, right=False)


In [32]:
df

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,postal_code,price_category
0,480,NaN,1993,manual,75.0,golf,150000,0,petrol,volkswagen,NaN,70435,0-500
1,18300,coupe,2011,manual,NaN,NaN,125000,5,gasoline,audi,yes,66954,18000-18500
2,9800,suv,2004,auto,163.0,grand,125000,8,gasoline,jeep,NaN,90480,9500-10000
3,1500,small,2001,manual,75.0,golf,150000,6,petrol,volkswagen,no,91074,1500-2000
4,3600,small,2008,manual,69.0,fabia,90000,7,gasoline,skoda,no,60437,3500-4000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
333030,3200,sedan,2004,manual,225.0,leon,150000,5,petrol,seat,yes,96465,3000-3500
333032,2200,NaN,2005,NaN,NaN,NaN,20000,1,NaN,sonstige_autos,NaN,39576,2000-2500
333033,1199,convertible,2000,auto,101.0,fortwo,125000,3,petrol,smart,no,26135,1000-1500
333034,9200,bus,1996,manual,102.0,transporter,150000,3,gasoline,volkswagen,no,87439,9000-9500


Проведём небольшое исследование 🔍 самую малость погуглив мы можем убедиться в том, что имеющиеся в нашем распоряжении индексы - это индексы Германии:

Причём значений с 4-й разрядностью гораздо меньше, чем с 5-й, оно и не удивительно, ведь причина меньшей разрядности в потерянном нуле в самом начале числа, так как мы имеем дело не просто с какими-то там абстрактными индексами, а системой ```PLZ``` (нем. Postleitzahl) — современной системой почтовых индексов Германии. О системе организации почтовой индексации в Германии можно почитать в [Википедии](https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D1%87%D1%82%D0%BE%D0%B2%D1%8B%D0%B5_%D0%B8%D0%BD%D0%B4%D0%B5%D0%BA%D1%81%D1%8B_%D0%B2_%D0%93%D0%B5%D1%80%D0%BC%D0%B0%D0%BD%D0%B8%D0%B8) или на сайте [auto.germany.ru](http://auto.germany.ru/map.html)

In [33]:
df.sample(10, random_state=2)

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,postal_code,price_category
292883,11250,sedan,2007,auto,184.0,c_klasse,150000,5,petrol,mercedes_benz,no,16348,11000-11500
42336,6300,sedan,2007,auto,140.0,a_klasse,150000,5,gasoline,mercedes_benz,no,88069,6000-6500
298222,650,sedan,1996,manual,75.0,golf,150000,9,petrol,volkswagen,no,36199,500-1000
81393,1350,other,1996,auto,103.0,3er,150000,11,petrol,bmw,NaN,84130,1000-1500
119165,7500,sedan,2008,manual,110.0,golf,150000,10,gasoline,volkswagen,yes,6484,7500-8000
267559,16995,convertible,2007,manual,200.0,tt,80000,3,petrol,audi,no,94522,16500-17000
325212,699,NaN,2016,NaN,60.0,fiesta,150000,0,NaN,ford,NaN,36251,500-1000
140927,2300,small,1998,auto,55.0,fortwo,80000,12,petrol,smart,no,26388,2000-2500
280329,4599,sedan,2003,auto,NaN,NaN,150000,11,gasoline,audi,no,13583,4500-5000
44013,4800,bus,2007,manual,90.0,transit,125000,11,gasoline,ford,no,82544,4500-5000


Добавил нули перед индексами состоящим из 4 символов. Нам это надо для обединения датафреймов далее.

In [34]:
df['postal_code'] = df['postal_code'].astype(str).str.zfill(5)

In [35]:
df.sample(10, random_state=2)

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,postal_code,price_category
292883,11250,sedan,2007,auto,184.0,c_klasse,150000,5,petrol,mercedes_benz,no,16348,11000-11500
42336,6300,sedan,2007,auto,140.0,a_klasse,150000,5,gasoline,mercedes_benz,no,88069,6000-6500
298222,650,sedan,1996,manual,75.0,golf,150000,9,petrol,volkswagen,no,36199,500-1000
81393,1350,other,1996,auto,103.0,3er,150000,11,petrol,bmw,NaN,84130,1000-1500
119165,7500,sedan,2008,manual,110.0,golf,150000,10,gasoline,volkswagen,yes,06484,7500-8000
267559,16995,convertible,2007,manual,200.0,tt,80000,3,petrol,audi,no,94522,16500-17000
325212,699,NaN,2016,NaN,60.0,fiesta,150000,0,NaN,ford,NaN,36251,500-1000
140927,2300,small,1998,auto,55.0,fortwo,80000,12,petrol,smart,no,26388,2000-2500
280329,4599,sedan,2003,auto,NaN,NaN,150000,11,gasoline,audi,no,13583,4500-5000
44013,4800,bus,2007,manual,90.0,transit,125000,11,gasoline,ford,no,82544,4500-5000


Отлично, теперь все индексы состоят из 5 цифр, как соответствует системе PLZ.

Импортируем датафрейм с базой данных содержащей все почтовые индексы мира.
[источник геоданных](http://download.geonames.org/export/zip/allCountries.zip)

In [36]:


base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/BtaNQa6MbhPC7A'  

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('downloaded_file.csv', 'wb') as f:   
    f.write(download_response.content)

data = pd.read_csv('downloaded_file.csv', on_bad_lines='skip', sep='\t')

In [37]:
data

,сountry,postal_code,lk2,lk3,lk4,lk5,lk6,lk7,lk8,lk9,lk10,lk11
0,AD,AD100,Canillo,Canillo,02,NaN,NaN,NaN,NaN,42.5833,1.6667,6.0
1,AD,AD200,Encamp,Encamp,03,NaN,NaN,NaN,NaN,42.5333,1.6333,6.0
2,AD,AD400,La Massana,La Massana,04,NaN,NaN,NaN,NaN,42.5667,1.4833,6.0
3,AD,AD300,Ordino,Ordino,05,NaN,NaN,NaN,NaN,42.6000,1.5500,6.0
4,AD,AD600,Sant Julià de Lòria,Sant Julià de Lòria,06,NaN,NaN,NaN,NaN,42.4667,1.5000,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1549944,ZA,9982,Luckhoff,NaN,NaN,NaN,NaN,NaN,NaN,-29.7500,24.7833,4.0
1549945,ZA,9986,Luckhoff,NaN,NaN,NaN,NaN,NaN,NaN,-29.4000,25.0167,NaN
1549946,ZA,9986,Koffiefontein,NaN,NaN,NaN,NaN,NaN,NaN,-29.4000,25.0167,4.0
1549947,ZA,9987,Koffiefontein,NaN,NaN,NaN,NaN,NaN,NaN,-29.4000,25.0167,4.0


Добавил скачивание с Яндекс Диска

Взглянем на данные

Оставим только Германию

In [38]:
data = data[data['сountry'] == 'DE']
data


,сountry,postal_code,lk2,lk3,lk4,lk5,lk6,lk7,lk8,lk9,lk10,lk11
102835,DE,01945,Guteborn,Brandenburg,BB,NaN,00,Landkreis Oberspreewald-Lausitz,12066,51.4167,13.9333,4.0
102836,DE,01945,Tettau,Brandenburg,BB,NaN,00,Landkreis Oberspreewald-Lausitz,12066,51.4333,13.7333,4.0
102837,DE,01945,Kroppen,Brandenburg,BB,NaN,00,Landkreis Oberspreewald-Lausitz,12066,51.3833,13.8000,4.0
102838,DE,01945,Grünewald,Brandenburg,BB,NaN,00,Landkreis Oberspreewald-Lausitz,12066,51.4000,14.0000,4.0
102839,DE,01945,Ruhland,Brandenburg,BB,NaN,00,Landkreis Oberspreewald-Lausitz,12066,51.4576,13.8664,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
119308,DE,99996,Menteroda,Thüringen,TH,NaN,00,Unstrut-Hainich-Kreis,16064,51.3076,10.5632,4.0
119309,DE,99996,Obermehler,Thüringen,TH,NaN,00,Unstrut-Hainich-Kreis,16064,51.2700,10.5975,4.0
119310,DE,99998,Mühlhausen/Thüringen,Thüringen,TH,NaN,00,Unstrut-Hainich-Kreis,16064,51.1917,10.5250,4.0
119311,DE,99998,Körner,Thüringen,TH,NaN,00,Unstrut-Hainich-Kreis,16064,51.2333,10.6000,4.0


Но при анализе данных мы выяснили что это все почтовые индесы Германии. Зачем нам нужны почтовые индексы других стран?

О системе организации почтовой индексации в Германии можно почитать в [Википедии](https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D1%87%D1%82%D0%BE%D0%B2%D1%8B%D0%B5_%D0%B8%D0%BD%D0%B4%D0%B5%D0%BA%D1%81%D1%8B_%D0%B2_%D0%93%D0%B5%D1%80%D0%BC%D0%B0%D0%BD%D0%B8%D0%B8) или на сайте [auto.germany.ru](http://auto.germany.ru/map.html)

![plz_germany.md.png](https://ltdfoto.ru/images/2023/06/08/plz_germany.md.png)

Удалим дубликаты

In [39]:
data = data.drop_duplicates(subset='postal_code')

Теперь обеденим два датафрейма по postal_code.

In [40]:
df_merged = df.merge(data[['postal_code', 'lk2', 'lk3', 'lk4', 'lk5', 'lk6', 'lk7', 'lk8', 'lk9', 'lk10', 'lk11']], on='postal_code', how='left')

In [41]:
df_merged

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,...,lk2,lk3,lk4,lk5,lk6,lk7,lk8,lk9,lk10,lk11
0,480,NaN,1993,manual,75.0,golf,150000,0,petrol,volkswagen,...,Stuttgart,Baden-Württemberg,BW,Regierungsbezirk Stuttgart,081,Stuttgart,08111,48.7667,9.1833,4.0
1,18300,coupe,2011,manual,NaN,NaN,125000,5,gasoline,audi,...,Pirmasens,Rheinland-Pfalz,RP,NaN,00,"Pirmasens, kreisfreie Stadt",07317,49.2071,7.5911,4.0
2,9800,suv,2004,auto,163.0,grand,125000,8,gasoline,jeep,...,Nürnberg,Bayern,BY,Regierungsbezirk Mittelfranken,095,Nürnberg,09564,49.4371,11.1194,4.0
3,1500,small,2001,manual,75.0,golf,150000,6,petrol,volkswagen,...,Herzogenaurach,Bayern,BY,Regierungsbezirk Mittelfranken,095,Landkreis Erlangen-Höchstadt,09572,49.5680,10.8856,4.0
4,3600,small,2008,manual,69.0,fabia,90000,7,gasoline,skoda,...,Frankfurt am Main,Hessen,HE,Regierungsbezirk Darmstadt,064,"Frankfurt am Main, Stadt",06412,50.1924,8.6753,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307078,3200,sedan,2004,manual,225.0,leon,150000,5,petrol,seat,...,Neustadt bei Coburg,Bayern,BY,Upper Franconia,094,Landkreis Coburg,09473,50.3298,11.1206,4.0
307079,2200,NaN,2005,NaN,NaN,NaN,20000,1,NaN,sonstige_autos,...,Stendal,Sachsen-Anhalt,ST,NaN,00,Landkreis Stendal,15090,52.6069,11.8587,4.0
307080,1199,convertible,2000,auto,101.0,fortwo,125000,3,petrol,smart,...,Oldenburg,Niedersachsen,NI,NaN,00,"Oldenburg (Oldenburg), Stadt",03403,53.1267,8.2384,4.0
307081,9200,bus,1996,manual,102.0,transporter,150000,3,gasoline,volkswagen,...,Kempten,Bayern,BY,Swabia,097,Kempten (Allgäu),09763,47.7177,10.2989,4.0


**Вывод:**
- Мы добавили в наш датафрейм очень полезную категориальные признаки c местоположением авто на територии Германии.

### Удалим пропуски

Попсмотрим на пропуски и их процентное соотношение от общего числа.

In [42]:
count_missing_values_percentage(df_merged)

                    Missing Values  Missing Percentage
price                            0            0.000000
vehicle_type                 18332            5.969722
registration_year                0            0.000000
gearbox                      14011            4.562610
power                        14633            4.765161
model                        14633            4.765161
kilometer                        0            0.000000
registration_month               0            0.000000
fuel_type                    22453            7.311704
brand                            0            0.000000
repaired                     55819           18.177170
postal_code                      0            0.000000
price_category                 220            0.071642
lk2                            117            0.038100
lk3                            117            0.038100
lk4                            117            0.038100
lk5                         124347           40.492961
lk6       

Удалим, столбец lk5 так как он содержит 40% процентов пропусков, это слишком много.

In [43]:
del df_merged['lk5']

Удалим, пропуски в столбцах от lk2 до lk10.

In [44]:
df_merged = df_merged.dropna(subset=['lk2'])

Так как природу пропусков нам достоверно узнать не имеется возможности, заполним их заглушкой.

In [45]:
columns_to_fill = ['vehicle_type','gearbox', 'model', 'fuel_type', 'repaired', 'lk11']
df_merged[columns_to_fill] = df_merged[columns_to_fill].fillna('unknown')

Заполним пропуски в столбце power медианной группируя по бренду, ценновой категории, и году регистрации. По столбцу model группировать мы не сможем тк со столбцом power у них пропуски в одних и тех же строках, и значение заполняется 0. 

In [46]:
df_merged['power'] = df_merged.groupby(["brand", 'price_category',"registration_year" ])['power'].transform(lambda x: x.fillna(x.median()))


In [47]:
df_merged = df_merged.dropna(subset=['power', 'price_category'])


In [48]:
count_missing_values_percentage(df_merged)

                    Missing Values  Missing Percentage
price                            0                 0.0
vehicle_type                     0                 0.0
registration_year                0                 0.0
gearbox                          0                 0.0
power                            0                 0.0
model                            0                 0.0
kilometer                        0                 0.0
registration_month               0                 0.0
fuel_type                        0                 0.0
brand                            0                 0.0
repaired                         0                 0.0
postal_code                      0                 0.0
price_category                   0                 0.0
lk2                              0                 0.0
lk3                              0                 0.0
lk4                              0                 0.0
lk6                              0                 0.0
lk7       

### Подготовка данных к машинному обучению

Делим на таргеты и признаки и на тестовую и обучающую выборку

In [49]:

y = df_merged['price']
X = df_merged.drop('price', axis=1)

#Разобьем данные сначала на обучающую выборку 75% тестовую 25%
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_STATE)

In [50]:
#сверим размерности
print("Обучающая выборка:", X_train.shape)
print("-"*30)
print("Тестовая выборка:", X_test.shape)
print("-"*30)
print("Обучающая выборка таргет:", y_train.shape)
print("-"*30)
print("Тестовая выборка таргет:", y_test.shape)
print("-"*30)

Обучающая выборка: (227952, 21)
------------------------------
Тестовая выборка: (75985, 21)
------------------------------
Обучающая выборка таргет: (227952,)
------------------------------
Тестовая выборка таргет: (75985,)
------------------------------


In [51]:
X_train.sample(5)

,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,...,price_category,lk2,lk3,lk4,lk6,lk7,lk8,lk9,lk10,lk11
300701,bus,2007,manual,131.0,galaxy,150000,9,gasoline,ford,yes,...,5000-5500,Geilenkirchen,Nordrhein-Westfalen,NW,053,Kreis Heinsberg,05370,50.9675,6.1176,4.0
134997,bus,2010,auto,163.0,s_max,150000,6,gasoline,ford,no,...,13500-14000,Regensburg,Bayern,BY,093,Regensburg,09362,49.0143,12.1427,4.0
240439,wagon,2000,manual,139.5,other,150000,3,gasoline,mercedes_benz,yes,...,500-1000,Hennstedt,Schleswig-Holstein,SH,00,Kreis Dithmarschen,01051,54.2833,9.1667,4.0
145574,sedan,2004,manual,101.0,focus,125000,5,petrol,ford,no,...,1500-2000,München,Bayern,BY,091,Kreisfreie Stadt München,09162,48.1500,11.5833,4.0
261139,unknown,2016,manual,102.0,3er,150000,10,petrol,bmw,no,...,500-1000,Altena,Nordrhein-Westfalen,NW,059,Märkischer Kreis,05962,51.2947,7.6734,4.0


In [52]:
column_types = df_merged.dtypes
print(column_types)

price                    int64
vehicle_type            object
registration_year        int64
gearbox                 object
power                  float64
model                   object
kilometer                int64
registration_month       int64
fuel_type               object
brand                   object
repaired                object
postal_code             object
price_category        category
lk2                     object
lk3                     object
lk4                     object
lk6                     object
lk7                     object
lk8                     object
lk9                    float64
lk10                   float64
lk11                    object
dtype: object


In [53]:
display(get_sorted_unique_values(df_merged, 'lk6'))

['00',
 '051',
 '053',
 '055',
 '057',
 '059',
 '064',
 '065',
 '066',
 '081',
 '082',
 '083',
 '084',
 '091',
 '092',
 '093',
 '094',
 '095',
 '096',
 '097']

In [54]:
df_merged['lk6'] = df_merged['lk6'].astype('int64')

In [55]:
#Выделим категориальные и числовые признаки
cat = list(X_train.select_dtypes(include='object').columns)
num = list(X_train.select_dtypes(include=['int64', 'float64']).columns)
print(cat)
num

['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired', 'postal_code', 'lk2', 'lk3', 'lk4', 'lk6', 'lk7', 'lk8', 'lk11']


['registration_year',
 'power',
 'kilometer',
 'registration_month',
 'lk9',
 'lk10']

In [56]:
#создаем pipeline с ColumnTransformer, TargetEncoder, StandardScaler
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', TargetEncoder(cols=cat))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num),
        ('cat', categorical_transformer, cat)
    ])

## Обучение моделей

Linear Regression

In [57]:
pipe_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

#задаем параметры для grid search
param_grid = { }

#создаем объект GridSearchCV
grid_search_lr = GridSearchCV(pipe_lr, param_grid=param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)

#обучаем модель на тренировочных данных
grid_search_lr.fit(X_train, y_train)

#выводим результаты кросс-валидации
print("Best cross-validation score: ", -grid_search_lr.best_score_)

Best cross-validation score:  2764.1081857251115


Удалил лишние блоки

In [58]:
cv_results_lr = grid_search_lr.cv_results_
print("Среднее время обучения LinearRegression:", cv_results_lr['mean_fit_time'])
print("Среднее время предсказания LinearRegression:", cv_results_lr['mean_score_time'])

Среднее время обучения LinearRegression: [3.68973556]
Среднее время предсказания LinearRegression: [0.30844488]


CatBoostRegression

In [ ]:


pipe_ct = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor(random_state=RANDOM_STATE))
])

# задаем параметры для random search
param_dist = {
    'regressor__learning_rate': uniform(0.01, 0.3),
    'regressor__n_estimators': range(100, 301, 100),
}

# создаем объект RandomizedSearchCV
random_search_cb = RandomizedSearchCV(pipe_ct, param_distributions=param_dist, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)

# обучаем модель на тренировочных данных
random_search_cb.fit(X_train, y_train)

# выводим лучшие параметры и результаты кросс-валидации
print("Best parameters: ", random_search_cb.best_params_)
print("Best cross-validation score: ", -random_search_cb.best_score_)

0:	learn: 4442.0682604	total: 83.1ms	remaining: 8.23s
1:	learn: 4374.0464385	total: 114ms	remaining: 5.61s
2:	learn: 4306.0322645	total: 146ms	remaining: 4.72s
3:	learn: 4240.7951250	total: 178ms	remaining: 4.27s
4:	learn: 4177.1036510	total: 210ms	remaining: 3.99s
5:	learn: 4115.3484595	total: 241ms	remaining: 3.78s
6:	learn: 4055.8077003	total: 272ms	remaining: 3.62s
7:	learn: 3999.3595630	total: 304ms	remaining: 3.5s
8:	learn: 3940.2820507	total: 336ms	remaining: 3.39s
9:	learn: 3883.5906508	total: 367ms	remaining: 3.3s
10:	learn: 3830.7436704	total: 397ms	remaining: 3.21s
11:	learn: 3778.0992709	total: 428ms	remaining: 3.14s
12:	learn: 3727.0293572	total: 460ms	remaining: 3.08s
13:	learn: 3677.6659691	total: 491ms	remaining: 3.01s
14:	learn: 3628.4907160	total: 520ms	remaining: 2.95s
15:	learn: 3582.9426369	total: 551ms	remaining: 2.89s
16:	learn: 3536.3153039	total: 580ms	remaining: 2.83s
17:	learn: 3490.6127750	total: 614ms	remaining: 2.79s
18:	learn: 3445.8132848	total: 652ms	re

In [ ]:
cv_results_cb = random_search_cb.cv_results_
print("Среднее время обучения Catboost:", cv_results_cb['mean_fit_time'].mean())
print("Среднее время предсказания Catboost:", cv_results_cb['mean_score_time'].mean())

LGBMRegressor

In [ ]:


pipe_lg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(random_state=RANDOM_STATE))
])

# задаем параметры для random search
param_dist = {
    'regressor__learning_rate': [0.1, 0.2],
    'regressor__n_estimators': randint(100, 400),
    'regressor__max_depth': randint(10, 16),
}

# создаем объект RandomizedSearchCV
random_search_lgbm = RandomizedSearchCV(pipe_lg, param_distributions=param_dist, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)

# обучаем модель на тренировочных данных
random_search_lgbm.fit(X_train, y_train)

# выводим лучшие параметры и результаты кросс-валидации
print("Best parameters: ", random_search_lgbm.best_params_)
print("Best cross-validation score: ", -random_search_lgbm.best_score_)



In [ ]:
cv_results_lgbm = random_search_lgbm.cv_results_
print("Среднее время обучения LGBM:", cv_results_lgbm['mean_fit_time'].mean())
print("Среднее время предсказания LGBM:", cv_results_lgbm['mean_score_time'].mean())

In [ ]:
random_search_cb.best_params_

## Анализ моделей

**Предварительный вывод:**
- Вырвалась вперед у нас модель от LGBM с неплохой скоростью обучения - 27.42 и предсказания - 2.4, плюс точность неплохая rmse - 1557.457
- Кэтбуст тоже показал неплохие результаты но скорость обучения гораздо хуже
- Создадим бэггинг из наших двух лучших моделей

In [ ]:
print(random_search_cb.best_params_)

In [ ]:
catboost = CatBoostRegressor(learning_rate=random_search_cb.best_params_['regressor__learning_rate'],
                             n_estimators=random_search_cb.best_params_['regressor__n_estimators'],
                             random_state=RANDOM_STATE)

lgbm = LGBMRegressor(learning_rate=random_search_lgbm.best_params_['regressor__learning_rate'],
                             n_estimators=random_search_lgbm.best_params_['regressor__n_estimators'],
                             max_depth=random_search_lgbm.best_params_['regressor__max_depth'],
                             random_state=RANDOM_STATE)


bagging = BaggingRegressor(base_estimator=lgbm, n_estimators=6)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('bagging', bagging)
])

param_dist = {}

random_search_bagging = RandomizedSearchCV(pipeline, param_distributions=param_dist, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, refit=True)

random_search_bagging.fit(X_train, y_train)

print("Best cross-validation score: ", -random_search_bagging.best_score_)



Избавился. Ваш пример не сработал, пришлось изменить ключи словаря random_search_cb.best_params_ на ожидаемые ключи для CatBoostRegressor

In [ ]:
cv_results_bagging = random_search_bagging.cv_results_
print("Среднее время обучения Bagging:", cv_results_bagging['mean_fit_time'].mean())
print("Среднее время предсказания Bagging:", cv_results_bagging['mean_score_time'].mean())

Вышел хороший скор 1549.882, но время обучения - 110.76 не быстро

In [ ]:
bg_test = random_search_bagging.best_estimator_.predict(X_test)
RMSE = mean_squared_error(y_test, bg_test, squared=False)
print('RMSE bagging на тестовых данных:', round(RMSE, 2))

In [ ]:
test_lg = random_search_lgbm.best_estimator_.predict(X_test)
RMSE = mean_squared_error(y_test, test_lg, squared=False)
print('RMSE LGBM на тестовых данных:', round(RMSE, 2))

Константная модель:

In [ ]:
dummy = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor',DummyRegressor(strategy="mean"))
])

dummy.fit(X_train, y_train)

test_model = dummy.predict(X_test)

RMSE = mean_squared_error(y_test, test_model, squared=False)

print('RMSE DummyRegressor равен:', round(RMSE, 2))

## Общий вывод:
- Наша модель адекватна и по итогу победителем стала модель LGBMRegressor
- Скорость обучения 27.42 с
- Скорость предсказания 2.47 с
- Точность предсказания RMSE на тесте 1533.88